In [1]:
from game.PokerSimple import PokerSimple
from AgentActorCriticRange_SimpleGame import Agent

2022-05-07 09:21:11.212991: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-07 09:21:11.213014: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
agent = Agent()
game= PokerSimple(0,1)

2022-05-07 09:21:13.578054: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-07 09:21:13.578090: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-07 09:21:13.578112: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cromi-Lenovo-V15-ADA): /proc/driver/nvidia/version does not exist
2022-05-07 09:21:13.578385: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#first we need a function that chooses actions for the q-player based on the observation
import pickle
import numpy as np

with open('q_simple', 'rb') as q_simple_file:
    q = pickle.load(q_simple_file)
def choose_action_q(observation):

    hole = observation[4]
    #sb:
    if observation[0] == 0:
        if observation[3] == 0: #preflop
            if observation[-10] ==-1: #first in
                action = np.argmax(q['sb'][hole]['preflop']['preflop_fi'])

            else: # bb must have gone allin, if sb has another action to perform
                action = np.argmax(q['sb'][hole]['preflop']['preflop_to_push'])


        else: #postflop
            board = observation[6]
            if observation[2] != 0: #bb did not push
                action = np.argmax(q['sb'][hole]['postflop'][board]['to_check'])
                if action == 0:
                    action =1
                else:
                    action =2
            else:
                action = np.argmax(q['sb'][hole]['postflop'][board]['to_push'])


    #bb
    else:
        if observation[3] == 0: #preflop
            if observation[-10] == 0.5: #sb completed
                action = np.argmax(q['bb'][hole]['preflop']['to_call'])
                if action == 0:
                    action =1
                else:
                    action =2
            else: #sb pushed
                action = np.argmax(q['bb'][hole]['preflop']['to_push'])

        else:
            board = observation[6]
            if observation[2]!= 0: #bb is first in
                action = np.argmax(q['bb'][hole]['postflop'][board]['postflop_fi'])
                if action == 0:
                    action =1
                else:
                    action =2
            else: #bb checked and sb pushed
                action = np.argmax(q['bb'][hole]['postflop'][board]['postflop_to_push'])

    return action






In [5]:
#agent.actor_preflop =create_model()
#agent.actor_postflop = create_model()
import copy
n_games = 1000
total_score = 0
score = 0
training_observations = []
training_policies = []

for i in range(1,n_games):
    if i %100 == 0:
        print("Episode", i)
        agent.print_strategy()
    done, next_to_act, observation = game.reset()
    agent.ranges_vil = [[],[]]
    agent.set_range_vil(0, [0, 4.5, 4, 0, game.hole_0, 1.5, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])
    agent.set_range_vil(1, [0, 4.5, 4, 0, game.hole_1, 1.5, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])
    score = 0
    agent.saw_flop =[False, False]
    #we can reduce the learning-time by 50% because we have two players. We always need pairs of start and terminal states. Once we have those we can perform a learning step
    # We need to play the game until the terminal state is known.
    while not game.done:
        next_to_act = game.next_to_act[0]
        game.create_observation(next_to_act)
        if next_to_act == 0:
            action = agent.choose_action(game.observations[next_to_act])
            if game.observations[next_to_act][6] != 0 and not agent.saw_flop[next_to_act]:

                hole_hero = game.hole_0 if next_to_act == 1 else game.hole_1
                agent.update_range_flop(1 - next_to_act, hole_hero, game.observations[next_to_act][6])

            agent.update_range_action(next_to_act-1, game.observations[next_to_act], action)

            game.implement_action(next_to_act,action)
            rewards = agent.simulate(game, 10)

            agent.memory.store_transition(game.observations[next_to_act],action,rewards[next_to_act])

            #agent.learn(game.observations[next_to_act],action, rewards[next_to_act])
        else:
            observation = game.observations[1][:]
            observation.append(0) if observation[4] != observation[6] else observation.append(1)
            training_observations.append(observation)
            best_policy = [0,0,0]

            for run in range(10):
                best_reward = -10
                for act in range(3):

                    game_sim = copy.deepcopy(game)
                    game_sim.implement_action(1,act)
                    rewards = agent.simulate(game_sim, 5, next_to_act)

                    if rewards[1] > best_reward:
                        best_reward = rewards[1]
                        action_villain = act

                if game.observations[next_to_act][6] != 0 and not agent.saw_flop[next_to_act]:

                    hole_hero = game_sim.hole_1
                    agent.update_range_flop(1 - next_to_act, hole_hero, game.observations[next_to_act][6])
                best_policy[action_villain]+=1
            agent.update_range_action(next_to_act-1, game.observations[next_to_act], action_villain)
            training_policies.append(np.array(best_policy)/10)

            game.implement_action(1, action_villain)

    game.create_observation(0)
    stack = game.observations[0][1]
    score = stack -5
    total_score += score

    if i %10 == 0:
        print("episode", i, "total score", total_score, "avg score", total_score/i)


TypeError: simulate() takes 3 positional arguments but 4 were given

TypeError: simulate() takes 3 positional arguments but 4 were given

In [238]:
X_preflop = X[X[:,3]==0]
X_postflop = X[X[:,3]==1]
y_preflop = y[X[:,3]==0]
y_postflop = y[X[:,3]==1]

In [239]:

agent.actor_preflop(X_preflop)

<tf.Tensor: shape=(461, 3), dtype=float32, numpy=
array([[5.3152126e-01, 4.2783204e-01, 4.0646620e-02],
       [4.2253676e-01, 2.2594966e-01, 3.5151356e-01],
       [1.4868068e-02, 7.4835426e-01, 2.3677772e-01],
       ...,
       [9.4736689e-01, 5.1973961e-02, 6.5911008e-04],
       [4.7047874e-01, 2.1920784e-01, 3.1031334e-01],
       [4.1695595e-01, 2.3439366e-01, 3.4865040e-01]], dtype=float32)>

In [240]:
y_preflop

array([[0.  , 1.  , 0.  ],
       [0.75, 0.25, 0.  ],
       [0.  , 0.95, 0.05],
       ...,
       [0.  , 1.  , 0.  ],
       [0.45, 0.  , 0.55],
       [0.8 , 0.  , 0.2 ]])

In [229]:
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from keras import backend as K

def create_network(layer_sizes, lr):
    model = keras.Sequential()

    for size in layer_sizes:
        model.add(keras.layers.Dense(size))
        model.add(keras.layers.LeakyReLU(alpha=0.05))

    model.add(keras.layers.Dense(3, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=lr), loss='mse')

    return model

In [312]:
def treesearch(gamestate, agent, n_sim):
        ranges = copy.copy(agent.ranges_vil) #save to return to agent. We don't want to copy entire anget-object.
        saw_flop = copy.copy(agent.saw_flop)
        print("run new search")
        #terminal condiation
        if gamestate.done:
            ev_0 = gamestate.stacks[0] -4.5 if gamestate.position_0 == 0 else gamestate.stacks[0] -4
            ev_1 = gamestate.stacks[1] -4.5 if gamestate.position_1 == 0 else gamestate.stacks[1] -4
            return ev_0, ev_1, None


        next = gamestate.next_to_act[0]
        gamestate.create_observation(next)
        state = gamestate.observations[next][:]
        legal_actions = []
        if state[1] != state[2]:
            legal_actions.append(0)
        legal_actions.append(1)
        if state[2]!= 0:
            legal_actions.append(2)
        action_count = {action: 0 for action in legal_actions}
        evs = [{action: 0 for action in legal_actions}, {action:0 for action in legal_actions}]

        for i in range(n_sim):
            game = copy.deepcopy(gamestate)
            policy = agent.actor
            action = agent.choose_action(state)
            while action not in legal_actions:
                action = agent.choose_action(state)

            action_count[action] += 1
            agent.update_range_action(1- next,state, action)
            game.implement_action(next,action)
            while not game.done:
                ev_0, ev_1, action_next = treesearch(game,agent, n_sim)
                game.create_observation(game.next_to_act)
                agent.update_range_action(1- game.next_to_act[0], game.observations[game.next_to_act[0]], action_next)

            ev_0 = game.stacks[0] -4.5 if game.position_0 == 0 else game.stacks[0]-4
            ev_1 = game.stacks[1] -4.5 if game.position_1 == 0 else game.stacks[1]-4
            if next ==0:
                evs[next][action] += ev_0
                evs[1-next][action] += ev_1
            else:
                evs[next][action] += ev_1
                evs[1-next][action] += ev_0


        best_action = max(evs[next], key=evs[next].get)
        ev_0 = evs[0][best_action]
        ev_1 = evs[1][best_action]
        agent.ranges_vil = ranges
        return ev_0, ev_1, best_action

In [313]:
game.reset()
treesearch(game, agent, 1)

run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new search
run new se

KeyboardInterrupt: 

In [303]:
evs

[{0: 0, 1: 0, 2: 0}, {0: 0, 1: 0, 2: 0}]

In [267]:
game.create_observation(0)
game.observations[0]

[1, 4, 4, 0, 3, 2, 0, 0.5, -1, -1, -1, -1, -1, -1, -1, -1, -1]

In [282]:
legal_actions = [0,1,2]
evs = [{action: 0 for action in legal_actions}, {action:0 for action in legal_actions}]
evs[0]

[{0: 0, 1: 0, 2: 0}, {0: 0, 1: 0, 2: 0}]

In [286]:
action_count

{0: 0, 1: 0, 2: 0}

In [5]:
import pickle
import numpy as np

with open('q_simple', 'rb') as q_simple_file:
    q = pickle.load(q_simple_file)
def choose_action_q(observation):

    hole = observation[4]
    #sb:
    if observation[0] == 0:
        if observation[3] == 0: #preflop
            if observation[-10] ==-1: #first in
                action = np.argmax(q['sb'][hole]['preflop']['preflop_fi'])

            else: # bb must have gone allin, if sb has another action to perform
                action = np.argmax(q['sb'][hole]['preflop']['preflop_to_push'])


        else: #postflop
            board = observation[6]
            if observation[2] != 0: #bb did not push
                action = np.argmax(q['sb'][hole]['postflop'][board]['to_check'])
                if action == 0:
                    action =1
                else:
                    action =2
            else:
                action = np.argmax(q['sb'][hole]['postflop'][board]['to_push'])


    #bb
    else:
        if observation[3] == 0: #preflop
            if observation[-10] == 0.5: #sb completed
                action = np.argmax(q['bb'][hole]['preflop']['to_call'])
                if action == 0:
                    action =1
                else:
                    action =2
            else: #sb pushed
                action = np.argmax(q['bb'][hole]['preflop']['to_push'])

        else:
            board = observation[6]
            if observation[2]!= 0: #bb is first in
                action = np.argmax(q['bb'][hole]['postflop'][board]['postflop_fi'])
                if action == 0:
                    action =1
                else:
                    action =2
            else: #bb checked and sb pushed
                action = np.argmax(q['bb'][hole]['postflop'][board]['postflop_to_push'])

    return action

In [164]:
agent.print_strategy()

--------------------------------------
preflop

sb
 first in

holecard 1 value 0.37521458 probs [0.738, 0.125, 0.137]
holecard 2 value 0.2781134 probs [0.603, 0.247, 0.15]
holecard 3 value 0.16249543 probs [0.421, 0.479, 0.1]
holecard 4 value 0.07969345 probs [0.245, 0.676, 0.08]
holecard 5 value 0.019323044 probs [0.0, 0.952, 0.047]
preflop

sb
 complete - push

holecard 1 value 0.2976578 probs [0.998, 0.002, 0.0]
holecard 2 value 0.29608223 probs [0.1, 0.899, 0.0]
holecard 3 value 0.28853095 probs [0.032, 0.968, 0.0]
holecard 4 value 0.27919874 probs [0.0, 1.0, 0.0]
holecard 5 value 0.26711118 probs [0.0, 1.0, 0.0]

bb
  to complete

holecard 1 value 0.40714088 probs [0.125, 0.07, 0.805]
holecard 2 value 0.36683857 probs [0.217, 0.098, 0.686]
holecard 3 value 0.28412345 probs [0.363, 0.119, 0.518]
holecard 4 value 0.19838674 probs [0.6, 0.141, 0.259]
holecard 5 value 0.13824359 probs [0.737, 0.147, 0.116]

bb
  to push

holecard 1 value 0.22092569 probs [0.856, 0.143, 0.002]
holecard

In [48]:
situations_q

{'pre_sb_first_in': 5073.0,
 'pre_sb_complete_to_push': 0,
 'pre_bb_to_call': -2029.0,
 'pre_bb_to_push': 0,
 'post_sb_to_check': 15751.0,
 'post_sb_to_push': -5232.0,
 'post_bb_first_in': -4844.0,
 'post_bb_check_pushsb': -10201.0}

In [252]:
import copy
game = PokerSimple(0,1)
n_games = 50_000
total_score = 0
score = 0
situations_q = {'pre_sb_first_in':0, 'pre_sb_complete_to_push':0,'pre_bb_to_call': 0, 'pre_bb_to_push':0,
                'post_sb_to_check':0, 'post_sb_to_push':0, 'post_bb_first_in':0, 'post_bb_check_pushsb':0}
training_observations = []
for i in range(1,n_games):
    if i%10 == 0:
        print("Episode", i)
    done, next_to_act, observation = game.reset()
    action_hero = -1
    situations = []
    while not game.done:
        next_to_act = game.next_to_act[0]

        if next_to_act == 0:

            game.create_observation(0)
            observation = game.observations[0]
            action_hero = agent.choose_action(observation)

            best_reward = -5
            for act in range(3):
                #print("trying action", act)
                game_sim = copy.deepcopy(game)
                game_sim.implement_action(0,act)
                rewards = agent.simulate(game_sim, 10)
                #print("ev:", rewards[0])
                if rewards[0] > best_reward:
                    best_reward = rewards[0]
                    action_hero = act

            game.implement_action(0, action_hero)

        else:
            game.create_observation(1)

            if game.observations[1][0]==0: #sb
                if game.observations[1][3]== 0: #preflop
                    if game.observations[1][1] == 4.5: #sb first in
                        situation = 'pre_sb_first_in'
                    else: #sb completed and bb pushed
                        situation = 'pre_sb_complete_to_push'
                else:#postflop
                    if game.observations[1][2] == 0: #bb pushed
                        situation = 'post_sb_to_push'
                    else:
                        situation = 'post_sb_to_check'

            else: #bb
                if game.observations[1][3]== 0: #preflop
                    if game.observations[1][1] == 4:
                        situation = 'pre_bb_to_call'
                    else:
                        situation = 'pre_bb_to_push'
                else:#postflop
                    if game.observations[1][2] != 0:#bb first in
                        situation = 'post_bb_first_in'
                    else:
                        situation =  'post_bb_check_pushsb'



            action = choose_action_q(game.observations[1])
            game.implement_action(1, action)

        situations.append(situation)

    game.create_observation(0)
    stack = game.observations[0][1]
    score = stack -5
    total_score += score
    for move in situations:
        situations_q[move] -= score

    if i %10 == 0:
        print("episode", i, "total score", total_score, "avg score", total_score/i)

Episode 10
episode 10 total score -2.75 avg score -0.275
Episode 20
episode 20 total score -1.5 avg score -0.075
Episode 30
episode 30 total score 4.0 avg score 0.13333333333333333
Episode 40
episode 40 total score 7.5 avg score 0.1875
Episode 50
episode 50 total score 10.5 avg score 0.21
Episode 60
episode 60 total score 21.5 avg score 0.35833333333333334
Episode 70
episode 70 total score 25.5 avg score 0.36428571428571427
Episode 80
episode 80 total score 27.75 avg score 0.346875
Episode 90
episode 90 total score 30.5 avg score 0.3388888888888889
Episode 100
episode 100 total score 37.25 avg score 0.3725
Episode 110
episode 110 total score 37.25 avg score 0.3386363636363636
Episode 120
episode 120 total score 39.5 avg score 0.32916666666666666


KeyboardInterrupt: 